In [20]:
from tqdm import tqdm
from pypdf import PdfReader

DOC_TITILE = "Compa/1) BUD Funding Agreement_53-2023-12-0484-FA-N"
TRANSLATION_ERROR = "XXX"
FONT_SIZE = 10
LEFT_MARGIN = 30
TOP_MARGIN = 30
N_BREAKLINE_CHARS = 80
LINE_HEIGHT = 15
DOUBLE_LINE_HEIGHT = 22
IS_DOUBLE_COLUMN = False
DEBUG = False

reader = PdfReader(F"data/{DOC_TITILE}.pdf")
pdf_texts = [p.extract_text() for p in reader.pages]

In [21]:
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)


In [22]:
print (word_wrap(pdf_texts[10]))

-  11  - 
 
payments, charges and expenses on a full indemnity basis)
suffered or incurred 
by an Indemnified Party; and (ii) all action,
proceedings, claims, and demands 
threatened, brought or institut ed
against an Indemnified Party (whether or not 
successful, compromised,
settled, withdrawn or discontinued) (collectively, 
“Third Party
Claims”); and everything mentioned in (i) above due to such Third

Party Claims, which in any case arise directly or indirectly in
connection with, 
out of or in relation to:  
 
(a) any breach of the
Terms of the Project by the Grantee, regardless of 
whether such breach
was caused by the Grantee or any member of the 
Project Team; or  


(b) any act or omission or negligence  or default of the Grantee or
any 
member of the Project Team in the implementation of the Project;
or  
 
(c) any allegation or claim that the use, operation or
possession of the Project 
Deliverables or the exercise by an
Indemnified Party of any right granted 
under the Ter

In [23]:
from deep_translator import GoogleTranslator
import time

translator = GoogleTranslator(source='en', target='vi')
def translate(text):
    num_try = 1000
    while num_try > 0:
        try:
            return translator.translate(text)
        except:
            num_try -= 1
            time.sleep(1)
    # return translator.translate(text)
    return TRANSLATION_ERROR



In [24]:
def split_text_to_paragraphs(text):
    paragraphs = []
    texts = text.split('\n')
    running_text = ''
    for id, text in enumerate(texts):
        print (id, text)
        if len(text) == 0:
            continue
        if text[0].isupper() or text[0].isdigit() or text[0] == '(' or (id > 0 and texts[id-1][-1] in ['.', '?', '!', ':']):
            if running_text:
                paragraphs.append(running_text)
            running_text = text
        else:
            running_text += ' ' + text

    paragraphs.append(running_text)

    # print (paragraphs)
    return paragraphs

# paragraphs = split_text_to_paragraphs(pdf_texts[28])

In [25]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont



def create_new_page(lines=["cong hoa xa hoi chu nghia viet nam"], end_paragraphs=[1]):
    if len(lines) == 0:
        lines = ["empty"]

    # width, height = letter  # Get the dimensions of the page
    # Get the page size
    page_size = reader.pages[0].mediabox
    width = page_size[2]
    height = page_size[3]

    left_margin = LEFT_MARGIN
    top_margin = TOP_MARGIN
    line_height = LINE_HEIGHT  # The height of each line

    # Calculate the x and y coordinates
    x = left_margin
    y = height - top_margin  # Subtract the top margin from the height

    # Register a font that supports Vietnamese characters
    pdfmetrics.registerFont(TTFont('DejaVuSans', 'DejaVuSans.ttf'))

    # Create a new PDF with the text
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=(width, height))
    can.setFont('DejaVuSans', FONT_SIZE)

    for line_id, line in enumerate(lines):
        print (line)
        encoded_line = line.encode('utf-8')
        can.drawString(x, y, encoded_line.decode('utf-8'))
        y -= line_height
        if (line_id + 1) in end_paragraphs:
            y -= -line_height + DOUBLE_LINE_HEIGHT

        #double column
        if line_id == len(lines) // 2 - 1 and IS_DOUBLE_COLUMN:
            x = width / 2 + left_margin // 2
            y = height - top_margin

    can.save()

    # Move the buffer position to the beginning
    packet.seek(0)
    return packet

In [26]:
from pypdf import PdfWriter
writer = PdfWriter()


In [27]:
def break_line(text, n_chars=N_BREAKLINE_CHARS):
    lines = []
    # if len(text) == 0: return lines
    print ('text', text)
    words = text.split(' ')
    print ('words', words)
    
    running_text = ''
    for word in words:
        if len(running_text) + len(word) < n_chars:
            running_text += ' ' + word
        else:
            lines.append(running_text)
            running_text = word
    lines.append(running_text)
    print (lines)
    return lines

def break_paragraphs(paragraphs):
    lines = []
    end_paragraphs = []
    for paragraph in paragraphs:
        lines += break_line(paragraph)
        end_paragraphs.append(len(lines))
    return lines, end_paragraphs


In [28]:
final_page_id = 0
end_page = 30 if DEBUG else len(pdf_texts)
for page_id, pdf_text in tqdm(enumerate(pdf_texts[:end_page])):
    if page_id < 15 or page_id > 17: continue
    writer.add_page(reader.pages[page_id])
        
    paragraphs = split_text_to_paragraphs(pdf_text)
    paragraphs_vi = [translate(paragraph) for paragraph in paragraphs]
    if TRANSLATION_ERROR in paragraphs_vi:
        print (f"ERROR at page {page_id}")
        break
    print (paragraphs_vi)
    paragraphs_vi = [p for p in paragraphs_vi if p is not None]
    # paragraphs_vi = '\n'.join(paragraphs_vi)
    paragraphs_vi, end_paragraphs = break_paragraphs(paragraphs_vi)
    for paragraph in paragraphs_vi:
        print (paragraph)
    print ("-------------------")

    packet = create_new_page(lines=paragraphs_vi, end_paragraphs=end_paragraphs)
    new_pdf = PdfReader(packet)
    writer.add_page(new_pdf.pages[0])
    final_page_id = page_id

# Write out the new PDF
with open(f"data/{DOC_TITILE}_translated_{final_page_id+1}.pdf", "wb") as out_file:
    writer.write(out_file)

1it [00:12, 12.70s/it]

['Quỹ chuyên dụng xây dựng thương hiệu, nâng cấp và bán hàng nội địa', '(Chương trình FTA và IPPA)', 'Hiệp định', 'Số đăng ký : 53-2023 -12-0484 -FA-N', 'Thỏa thuận này được lập vào ngày 31 tháng 1 năm 2024 giữa:', '(a) Hội đồng Năng suất Hồng Kông, một cơ quan theo luật định được thành lập theo Luật Hồng Kông', 'Pháp lệnh Hội đồng Năng suất Kong (Cap. 1116) của Hồng Kông có văn phòng đặt tại Tòa nhà HKPC, 78 Đại lộ Tat Chee, Cửu Long, Hồng Kông', '(“HKPC”)', '(b) CÔNG TY TNHH CÔNG NGHỆ COMPATHNION, doanh nghiệp đăng ký tại Hồng Kông', 'Kong theo Pháp lệnh Đăng ký Kinh doanh (Cap. 310) có trụ sở đăng ký tại UNIT 617 6/F BUILDING 12W, GIAI ĐOẠN 3, Công viên Khoa học Hồng Kông,', 'Lãnh thổ mới (“Người được tài trợ”).', 'TUYỆT VỜI', '(1) Chính phủ đã thành lập Quỹ chuyên trách xây dựng thương hiệu, nâng cấp và', 'Bán hàng nội địa (“Quỹ BUD”) (Chương trình FTA và IPPA) tại Hồng Kông (như được định nghĩa dưới đây) nhằm tài trợ cho các dự án và hoạt động hỗ trợ các doanh nghiệp Hồng Kông phá

2it [00:17,  7.86s/it]

['- 2 -', 'Chi phí dự án thực tế có nghĩa là đối với bất kỳ khoảng thời gian cụ thể nào, tức là . dù là 12 tháng đầu tiên của Thời hạn Dự án hay toàn bộ Thời hạn Dự án, tất cả và bất kỳ khoản chi nào cho các hạng mục chi tiêu được quy định trong Nội dung được phê duyệt', 'Đề xuất dự án (nhưng không bao gồm phí kiểm toán phải trả để mua sắm Tài khoản được kiểm toán) mà Bên nhận tài trợ thực tế phải chịu khi thực hiện Dự án và được kiểm toán trong các tài khoản được kiểm toán có liên quan (cho dù đó là', 'Tài khoản được kiểm toán hàng năm hoặc Tài khoản được kiểm toán cuối cùng).', 'Ngân sách được phê duyệt là ngân sách được Chính phủ phê duyệt được nêu trong Đề xuất dự án được phê duyệt.', 'Dự án được phê duyệt', 'Đề xuất là đề xuất dự án (có trong mẫu đơn đăng ký) được Chính phủ phê duyệt bao gồm Ngân sách được phê duyệt, được đính kèm tại', 'Phụ lục I kèm theo đây.', 'Tài khoản được kiểm toán có nghĩa là bất kỳ bản sao kê tài khoản nào liên quan đến', 'Dự án bao gồm tài khoản thu nhập

3it [00:21,  6.03s/it]

...

['- 31 -', '20.2 Bất chấp Điều khoản 20.1 ở trên, theo đây tuyên bố rằng Chính phủ với tư cách là người đứng đầu HKPC đối với Thỏa thuận này có thể thực thi tất cả các Điều khoản của Dự án bao gồm mọi điều khoản của Thỏa thuận này. Việc thực thi có thể bao gồm nhưng không giới hạn việc tiến hành các thủ tục pháp lý để thực thi tất cả và bất kỳ quyền và khiếu nại nào phát sinh từ hoặc liên quan đến hoặc liên quan đến', 'Điều khoản của Dự án. Để tránh nghi ngờ, trừ khi tòa án yêu cầu, Chính phủ có thể tiến hành bất kỳ thủ tục pháp lý nào chống lại Người được cấp mà không nêu tên HKPC là một trong các nguyên đơn. Theo những điều đã nói ở trên và', 'Điều 20.3 dưới đây, không có người nào khác, ai hoặc không phải là một bên của điều này', 'Thỏa thuận này, có thể thực thi bất kỳ Điều khoản nào của Dự án hoặc bất kỳ quy định nào trong thỏa thuận này', 'Thỏa thuận theo hoặc theo hoặc vì mục đích của Cap. 623 (bao gồm phần 4 của Điều 623).', '20.3 Không có nội dung nào trong Điều 20.2 ở trên nh

32it [03:44,  5.87s/it]

['- 32 -', 'NHƯ BẰNG CHỨNG, Thỏa thuận này được các Bên ký thành hai bản vào ngày và năm được ghi đầu tiên ở trên.', 'KÝ KẾT BỞI  }', 'Ông Chan Yee Wah, Ernest }', 'Tổng Giám đốc, Bộ phận Tài trợ Công nghệ } người đại diện được ủy quyền hợp pháp của }', 'Hội đồng Năng suất Hồng Kông } } } Ông Chan Yee Wah, Ernest } } trước sự chứng kiến \u200b\u200bcủa }', 'Ông Ip Yuk Kin, Valentine }', 'Trưởng phòng BUD, Bộ phận Tài trợ Công nghệ }', 'Hội đồng Năng suất Hồng Kông }', 'Ông Ip Yuk Kin, Valentine', 'KÝ KẾT BỞI  }', 'Mr. CHAN Chi Chuen  }', 'Giám đốc điều hành } thay mặt cho }', 'CÔNG TY TNHH CÔNG NGHỆ COMPATHNION } } Ông CHAN Chí Xuyên } } trước sự chứng kiến \u200b\u200bcủa }', 'Bà NGO Thị Thu Hà }', 'Nhân viên Marketing Việt Nam }', 'CÔNG TY TNHH CÔNG NGHỆ COMPATHNION } } (Công ty Chop của Người được cấp phép)', 'Bà NGO Thị Thu Hà']
text - 32 -
words ['-', '32', '-']
[' - 32 -']
text NHƯ BẰNG CHỨNG, Thỏa thuận này được các Bên ký thành hai bản vào ngày và năm được ghi đầu tiên ở trên.


33it [03:45,  6.82s/it]

['- 33 -', 'Phụ lục I', '(Đề xuất dự án đã được phê duyệt)']
text - 33 -
words ['-', '33', '-']
[' - 33 -']
text Phụ lục I
words ['Phụ', 'lục', 'I']
[' Phụ lục I']
text (Đề xuất dự án đã được phê duyệt)
words ['(Đề', 'xuất', 'dự', 'án', 'đã', 'được', 'phê', 'duyệt)']
[' (Đề xuất dự án đã được phê duyệt)']
 - 33 -
 Phụ lục I
 (Đề xuất dự án đã được phê duyệt)
-------------------
 - 33 -
 Phụ lục I
 (Đề xuất dự án đã được phê duyệt)
